## はじめに


適当に



## 目次
- [1.アダブースティング(Adaptive Boosting)](#sec1)  
- [2.特徴](#sec2)  
- [3.処理の流れ_概要](#sec3)  
- [4.処理の流れ_詳細](#sec4)  
- [5.発展 ](#sec5)  
- [6.まとめ](#sec6)  
- [7.参考文献](#sec7)  




<a name="sec1"></a>
## 1.アダブースティング(Adaptive Boosting)

アンサンブル学習法の１つ。
有名なアンサンブル学習法に「バギング」と「ブースティング」があるが、「ブースティング」の中で特に有名な手法が「アダブースティング」である。
有名ゆえに「ブースティング」と言ったら「アダブースティング」を指すことが多い。

ざっくり言うと、「アダブースティング」は、**「間違えた箇所をより重点的に学習する」ということを何回か繰り返すことで、弱学習器から強い学習器を作る手法。**
前回のモデルによる誤差を利用するので**「並列化はできない」**点に注意が必要。  
「Adaptive（適応できる）」というのは**「前回の学習結果を次の学習に反映できる」**くらいの意味だと解釈した。

また、回帰・分類のいずれにも適用可能で、弱学習器には「教師あり学習」を利用する。

<br>
<br>
<a name="sec2"></a>
## 2.特徴
1.事前のチューニングが不要<br>
2.ノイズの影響を受けやすい（過剰適合しやすい）

<br>
<br>

<a name="sec3"></a>
## 3.処理の流れ_概要
２値分類の場合で説明します。
アダブースティングでは「データの重みベクトル」と「弱識別器の重み（T次元）」の２つのベクトルが特徴的ですね。

1:	データと弱識別器（Z個）を用意する。  
2:	データの重みベクトルを初期化。（w<sub>1</sub>(x) = 1 / M）  
    （データの重みベクトルとは、次の学習でどれだけそのデータを重要視して学習させるか、というベクトルである。）
3:	データの重みベクトルがw<sub>t</sub>の状態で、Z個の弱識別器で学習を行う。  
4:	もっとも重み付き誤差が小さい各弱識別器(h<sub>t</sub>)を1つだけ選ぶ。  
     ここでの「重み付き」とは、データの重みベクトル(w<sub>t</sub>)を元に計算することである。  
5:	上記で選択した弱識別器の重み付き誤差を元に、取り出した弱識別器の重み(α<sub>t</sub>)を更新する。  
6:	データの重みベクトル(w<sub>t</sub>)を更新。  
7:	3-6を最初に指定した回数分(T回)繰り返す
8:	弱識別器の重み付き和を計算することで、強分類器を構築する。 


Z個の弱識別器とあるが、これは１つのアルゴリズムをZ個使ってもよいし、Z種類のアルゴリズムを利用しても良いのかな...?
（吉村がよくわかっていない）

<br>
<br>

<a name="sec4"></a>
## 4.処理の流れ_詳細
ちょっとつまづきそうなところだけ説明。


#### 4-1.弱識別器ごとの重み付き誤差の計算  
各反復回数での識別器の重み付き誤差を**$ε_t$**とした時、下記の計算式で求める。  
$\hat{y}$は弱識別器から予測されるラベル、yは正解のラベルです。

\begin{equation*}
ε_t = w_t (\hat{y} \neq y)
\end{equation*}

#### 4-2.弱識別器の重み(α<sub>t</sub>)  
上記で求めた「各反復回数ごとの弱識別器の重み付き誤差(**$ε_t$**)」を利用して計算する。

\begin{equation*}
 α_t = \frac{1}{2}\log \frac{1-ε_t}{ε_t}
\end{equation*}

誤分類率がちょうど50%だった場合はその弱識別器は最終的な識別器を求める段階では無視されるね。  
誤分類率が50%以上になると負の値をとる点に注意。  
精度が高い弱識別器はより強く優先される。


#### 4-3.データの重みベクトル(w<sub>t</sub>)の更新
弱識別器(h<sub>t</sub>)が誤分類したデータの重みベクトル(w<sub>t</sub>)を大きくし、正しく分類したデータの重みを小さくする。  
(次のステップで誤分類したデータをよりモデルに重点的に反映させるため)  


\begin{equation*}
 w :=  w * exp(-α_t  * \hat{y} * y)
\end{equation*}

正しく分類したデータについては、$\hat{y} * y$が正になり、誤分類したデータについては負になる。  
なので、次のステップで重みが大きくなるか、小さくなるかは「弱識別器の重みが正か負か」、「誤分類したか正しく分類したか」の２軸から判定している。

|  | 弱識別器の重みが正 | 弱識別器の重みが負 |
|:-----------|-----------:|:------------:|
| 誤分類 | $\nearrow$ | $\searrow$ |
| 正しく分類 | $\searrow$ | $\nearrow$ | 

ここで注目したい点として、**「アダブースティングでは精度が悪いモデルも有効活用する」**という点である。
精度が悪い（２値分類で精度が50%未満）モデルの場合、「弱識別器の重み」が負になる。この時誤分類したものについては「モデルを正しく選択すれば正しく分類できる」とポジティブに捉えているのである。


また、このデータの重みベクトルは最終的に正規化して合計が１になるようにする。

\begin{equation*}
 w := \frac{w}{\sum_{t=1}^{T}w}
\end{equation*}


#### 4-4.最終的な強識別器
最終的に求められる強学習器では、T個得られた弱学習器(h<sub>t</sub>)とそれらの重み(α<sub>t</sub>)から下記のような計算をする。

\begin{equation*}
H(x) = sign(\sum_{t=1}^{T} α_t * h_t(x))
\end{equation*}


この数式の意味は、各弱識別器の予測結果に重みをかけた結果の平均が0より大きければ正例、小さければ負例とする.といった意味である。

<br>
<br>

<a name="sec5"></a>
## 5.発展  
アダブースティングから発展したアルゴリズムについて。

#### 5-1.勾配ブースティング  
勾配降下法を使ったブースティング


#### 5-2.GBDT  
弱学習器に決定木を使った勾配ブースティング

<br>
<br>

<a name="sec6"></a>
## 6.まとめ
アダブースティングについて勉強したが、今回は２値分類のみに焦点を絞っていた。  
これを多クラス分類や回帰に使うとどのようになるかを勉強しよう。  
また、発展したアルゴリズムについてもまとめたい。
<br>
<br>


<a name="sec7"></a>
## 7.参考文献
- [Python機械学習プログラミング 達人データサイエンティストによる理論と実践](https://www.amazon.co.jp/Python%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0-%E9%81%94%E4%BA%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E3%81%AB%E3%82%88%E3%82%8B%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%B7%B5-impress-top-gear%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-ebook/dp/B01HGIPIAK/ref=sr_1_1?s=digital-text&ie=UTF8&qid=1514115340&sr=1-1&keywords=Python%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0+%E9%81%94%E4%BA%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E3%81%AB%E3%82%88%E3%82%8B%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%B7%B5)
- [アンサンブル法による機械学習：基礎とアルゴリズム](https://www.amazon.co.jp/dp/B073F12L1V/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1)

